In [1]:
from checkptr import Checkpointer
import torch
from torch import nn
from itertools import chain

In [2]:
class FFN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(FFN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))
  
model = FFN(10, 20, 1)

In [6]:
chkpt = Checkpointer.pytorch('checkpoints')
chkpt.checkpoint(model, 'model-2.pth')

In [8]:
chkpt.load('model-2.pth')

FFN(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)